<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/TwoTanksStateSpace2TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
import jax
jax.config.update('jax_enable_x64',True)
import jax.numpy as jnp
from plotly.subplots import make_subplots
from scipy.optimize import root
import scipy.signal as sig
from scipy.integrate import solve_ivp

In [7]:
alpha = 2.
Ac=2.

u_initial = jnp.array([2., 0.5, 0.4])

def rhs(t, h, u):
    h1, h2 = h
    u1, u2, u3 = u

    q1 = alpha * h1**0.5
    q2 = alpha * h2**0.5

    return jnp.array([(u1+u2-q1)/Ac, (q1+u3-q2)/Ac])




In [9]:
h_initial = root(lambda x: rhs(0., x, u_initial), [1., 1.]).x

In [14]:
Jh_func = jax.jacobian(rhs, 1)
Jh =Jh_func(0., h_initial, u_initial)
Jh

Array([[-0.4       ,  0.        ],
       [ 0.4       , -0.34482759]], dtype=float64)

In [16]:
Ju_func = jax.jacobian(rhs, 2)
Ju = Ju_func(0., h_initial, u_initial)
Ju

Array([[0.5, 0.5, 0. ],
       [0. , 0. , 0.5]], dtype=float64)

In [24]:
ltisys = sig.StateSpace(Jh, Ju, np.array([[0, 1.]]), np.array([[0.,0.,0.]]))
ltisys

StateSpaceContinuous(
array([[-0.4       ,  0.        ],
       [ 0.4       , -0.34482759]]),
array([[0.5, 0.5, 0. ],
       [0. , 0. , 0.5]]),
array([[0., 1.]]),
array([[0., 0., 0.]]),
dt: None
)

In [37]:
num, den = sig.ss2tf(Jh, Ju, np.array([[0, 1.]]), np.array([[0.,0.,0.]]),input =2 )
H2_U3TF= sig.TransferFunction(num, den)


/usr/local/lib/python3.10/dist-packages/scipy/signal/_filter_design.py:1746: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  warnings.warn("Badly conditioned filter coefficients (numerator): the "


In [39]:
t, h2dev = H2_U3TF.

In [40]:
fig = make_subplots()
fig.add_scatter(x=t, y=h2dev + h_initial[1])

In [46]:
res = solve_ivp(lambda t, h: rhs(t,h, u_initial + np.array([0,0,1.])), (0,20), h_initial, method='Radau',dense_output=True)

In [47]:
tplot = np.linspace(0,20, 100)
res.sol(tplot)

array([[1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.5625    , 1.5625    ,
        1.5625    , 1.5625    , 1.5625    , 1.56